In [6]:
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
import json
import os
from tqdm.notebook import tqdm,tnrange
import string
import numpy as np

nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to C:\Users\Sajeel
[nltk_data]     Khan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Sajeel
[nltk_data]     Khan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:

class InvertedIndex():
	def __init__(self):
		self.db={}

	def __str__(self):
		return "Keys:"+str(len(list(self.db.keys())))
	
	def stripSpecialChar(self,text):
		return ''.join(ch for ch in text if ch.isalnum() and not ch.isdigit() and ch not in string.punctuation)

	def preProcess(self,text):
		text = text.lower()
		text_tokens = word_tokenize(text)
		stopWords = set(stopwords.words('english'))
		validTokens = list(set(text_tokens) - set(stopWords))
		validTokens = [self.stripSpecialChar(x) for x in validTokens]
		validTokens = [x for x in validTokens if len(x) > 1]
		return validTokens

	def indexFile(self,file,fileId):
		tokens = self.preProcess(file)
		for i in tokens:
			if i in self.db:
				self.db[i].append(fileId)
			else:
				self.db[i] = [fileId]

	def save(self):
		json.dump(self.db, open("output.json", "w"))


In [8]:
allFiles = os.walk("stories")
filePaths = []
for i in allFiles:
	for j in i[2]:
		filePath = i[0] + "/" + j
		filePaths.append(filePath)

json.dump(filePaths, open("mapping.json", "w"))

index = InvertedIndex()

for i,filePath in enumerate(tqdm(filePaths)):
    try:
        file = open(filePath, encoding="utf8")
        read = file.read().replace('\n', ' ')    
    except Exception as e:
        file = open(filePath, encoding="Latin1")
        read = file.read().replace('\n', ' ')
    file.close()
    index.indexFile(read, i)
    index.save()

In [9]:
class Query():
    def __init__(self):
        pass

    def loadIndex(self,file):
        self.db=json.load(open(file))

    def OR(self,term1,term2):
        return term1.add(term2)
    
    def AND(self,term1,term2):
        return term1.intersect(term2)
    
    def ANDNOT(self,term1,term2):
        # temp=Response(set(sum(list(self.db.copy().values()),[])))
        temp=Response(np.arange(468))
        temp1=temp.diff(term2)
        return term1.intersect(temp1)
    def ORNOT(self,term1,term2):
        # temp=Response(set(sum(list(self.db.copy().values()),[])))
        temp=Response(np.arange(468))
        temp1=temp.diff(term2)
        temp2 = term1.intersect(term2)
        return temp1.add(temp2)


In [10]:
class Response():
    def __init__(self,data):
        self.data=set(data)

    def getMapping(self,file):
        self.mapping=json.load(open(file))
        for i in self.data:
            print(self.mapping[i])
    
    def __str__(self):
        return str(len(self.data))

    def add(self,response):
        return Response(set.union(self.data,response.data))
        
    def intersect(self,response):
        return Response(set.intersection(self.data,response.data))

    def diff(self,response):
        return Response(set.difference(self.data,response.data))
        

In [11]:
query=Query()
query.loadIndex("output.json")

In [12]:
queries=["stink","middle"]
output=Response(query.db[queries[0]])
for i in tnrange(1,len(queries)):
    curr=Response(query.db[queries[i]])
    output=query.AND(curr,output)

print(output)


11


In [13]:
output.getMapping("mapping.json")
output.data

stories/darkness.txt
stories/friends.txt
stories/long1-3.txt
stories/valen
stories/nihgel_8.9
stories/elite.app
stories/grav
stories/gulliver.txt
stories/forgotte
stories/sick-kid.txt
stories/hellmach.txt


{129, 145, 183, 193, 210, 214, 221, 270, 303, 378, 431}

In [14]:
def no_comparisonsOR(term1, term2):
    first = list(set(query.db[term1]))
    first.sort()
    second = list(set(query.db[term2]))
    second.sort()
    i=0
    j=0
    count=0
    ans = []
    while(i<len(first) and j<len(second)):
        count+=1
        if(first[i]<second[j]):
            ans.append(first[i])
            i+=1          
        elif(first[i]>second[j]):
            ans.append(second[j])
            j+=1
        else:
            ans.append(first[i])
            i+=1
            j+=1
    while(i<len(first)):
        ans.append(first[i])
        i+=1
    while(j<len(second)):
        ans.append(second[j])
        j+=1
    return count    

In [15]:
def no_comparisonsAND(term1, term2):
    first = list(set(query.db[term1]))
    first.sort()
    second = list(set(query.db[term2]))
    second.sort()
    i=0
    j=0
    count=0
    ans = []
    while(i<len(first) and j<len(second)):
        count+=1
        if(first[i]<second[j]):
            i+=1          
        elif(first[i]>second[j]):
            j+=1
        else:
            ans.append(first[i])
            i+=1
            j+=1
    return count

print(no_comparisonsAND("stink","middle"))

123


In [19]:
def no_comparisonsANDNOT(term1, term2):
    temp=Response(np.arange(468)) 
    second = list(set(query.db[term2])) #list of term2
    second.sort()
    second=temp.diff(second) #generating Universal - (Not terms2)
    second.sort() #sorting, if needed
    first = list(set(query.db[term1])) #list of term1
    first.sort()
    #initiating general AND operator
    i=0
    j=0
    count=0
    ans = []
    while(i<len(first) and j<len(second)):
        count+=1
        if(first[i]<second[j]):
            i+=1          
        elif(first[i]>second[j]):
            j+=1
        else:
            ans.append(first[i])
            i+=1
            j+=1
    return count

print(no_comparisonsANDNOT("telephone", "paved"))

AttributeError: 'list' object has no attribute 'data'

In [17]:
def no_comparisonsORNOT(term1, term2):
    temp=Response(np.arange(468)) 
    second = list(set(query.db[term2])) #list of term2
    second.sort()
    second=temp.diff(second) #generating Universal - (Not terms2)
    second.sort() #sorting, if needed
    first = list(set(query.db[term1])) #list of term1
    first.sort()
    i=0
    j=0
    count=0
    ans = []
    while(i<len(first) and j<len(second)):
        count+=1
        if(first[i]<second[j]):
            ans.append(first[i])
            i+=1          
        elif(first[i]>second[j]):
            ans.append(second[j])
            j+=1
        else:
            ans.append(first[i])
            i+=1
            j+=1
    while(i<len(first)):
        ans.append(first[i])
        i+=1
    while(j<len(second)):
        ans.append(second[j])
        j+=1
    return count  
    
    

In [18]:
print(no_comparisonsORNOT("telephone", "paved"))

AttributeError: 'list' object has no attribute 'data'